# BBC - Project: Analyse lexicale


Pour cette partie du projet, notre objectif était de regarder si on peut sélectionner des gènes à pâtir de la documentation existante.<br>

Pour faire ceci, nous avons décidé de calculer la similarité entre la description des gènes et une requête contentant des mots en rapport avec le cancer colorectal.<br>

Cette analyse se fait en 5 étapes:
- Extraction des gb_acc (gb access number) de chacun des gènes depuis le fichier de donnée.
- Recherche du résumée (Entrezgene_summary) de chacun des gènes en utilisant le parser Bio.Entrez
- Définir une requête (tableau de mots) qui sera le vecteur de requête
- Vectoriser les résumés de chacun des gènes en utilisant le tf-idf (cela permet de donner un poids moins important aux mots qui apparaissent souvent). Nous avons utilisé la librairie NLTK pour faire ceci.
- Calculer la similarité entre les vecteurs des documents et le vecteur de requête. Cela donnera des résultats entre 0 et 1 (0 = pas de similarité / pas de documentation, 1=timilarité totale).

Une fois ces étapes effectuées, nous obtenons un tableau de chacun des gènes avec un score de similarité. Il nous reste plus qu'à sélectionner les 400 gènes avec le meilleur score (nous avons choisi 400 pour être en accord avec l'analyse statistique). Ils ont été sauvegardé dans le fichier gene_similartites_400.npy


Nous aurions voulu tester l'efficacité de cette solution en regardant le pourcentage de gènes similaires entre les 2 méthodes. Si ce pourcentage est haut, cela signifie que l'analyse lexicale est pertinente (l'analyse lexicale avec l'analyse statistique). Cependant, nous n’avons pas eu le temps de faire cela.


### Difficultés et problèmes rencontrés

Le premier problème fut que le parser Entrez utilise les *gi* et que nous avions que les *gb_acc* comme identifiant. Nous avons donc dû trouver un moyen pour convertir les *gb_acc* en gi en utilisant le parser. Nous avons réussi à convertir une bonne partie des *id* mais pas tous. <br>
Un second problème est que Entrez limite le nombre d'appel. Nous avons dû mettre un sleep(3) à chaque fois que le serveur nous revoyait une erreur "connexion_timout". <br>
Nous avons résusi parser **37'023** gène sur les 50'000. <br>
L'énorme quantité de données était, la lenteur et l'instabilité du parseur fût le plus gros problème. Le code a planté plusieurs fois pendant l'exécution. Et comme une exécution prend au moins 12h (juste pour convertir le gb_acc en gi).<br>

### Ce qui reste à faire
Comparer les résultats de l'analyse lexicale avec l'analyse statistique pour voir si l'analyse lexicale est pertinente.

### Conclusion
Nous ne pensons pas qu'une analyse lexicale soit pertinente pour les raisons suivantes:
- Tous les gènes ne possèdent pas forcément une description.
- Cela prend beaucoup trop de temps d'exécution.
- Il n'est pas possible de trouver l'ig de certains gènes. Donc ils seront exclus de base même s'ils ont un lien.

Hormis ça, d'un point de vue purement "théorie du langage", les meilleurs résultats sont plutôt bon avec une similarité de 0.876.

C'était intéressant de faire ce travail. Mais le temps à disposition était trop limité. Le parseur nous a fait perdre énormément de temps. Mais nous pensons que la méthodologie employée est correcte.<br> Nous sommes tout de même un peu déçu de ne pas avoir pu faire l'analyse finale qui aurait pu dire si 'analyse lexicale est pertinente ou non.




## Implementation

Les cellules qui prennent trop de temps d'exécution ou d'écriture dans les fichiers ont été désactivées.

les requirement se trouvent dans le fichier: requirerments.txt

### Récupération des gb_acc dans le fichier de données

In [1]:
DATA_PATH = './data/'

In [2]:
import numpy as np
from Bio import Geo
from Bio import Entrez
import re
import os, codecs 
import nltk
from nltk.text import TextCollection
from math import*

def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records

In [ ]:
# re-loading the data
records = load_geo(DATA_PATH + 'GSE21510_family.soft')
series_sample_id = []
sample_titles = []
genes = []
gene_ids = []
nb_cols = 0
nb_rows = 0
data = []
for r in records:
    rea = r.entity_attributes
    if 'Series_geo_accession' in rea:
        if rea['Series_geo_accession'] == 'GSE21510':
            series_sample_id = rea['Series_sample_id']
            nb_cols = len(series_sample_id)
    if 'Sample_title' in rea:
        sample_titles.append(rea['Sample_title'])
        if 'sample_table_begin' in rea:
            nb_rows = rea['Sample_data_row_count'] 
            data.append(r.table_rows)
    if 'platform_table_begin' in rea:
        print('found')
        gene_ids.append(r.table_rows)

In [ ]:
# the keys of this dict are gene IDs, the values are GB_ACCs
genes_dict = {}
for i, g in enumerate(gene_ids[0]):
    if(i != 0): # do not inclide labels ('ID' & 'GB_ACC')
        genes_dict[g[0]] = g[1]

len(genes_dict)

### Récupération de la documentation

In [7]:
# Séparateurs pour l'écriture dans les fichiers
LINE_SEPARATOR = "$$$"
CONETENT_SEPARATOR = ":::"

In [12]:
# Chemins d'accès pour les fichiers de données
data_path = "C:\\Users\\Jeremie Chatillon\\Documents\\HEIG\\BA-06\\BBC\\BBC-projet\\data\\"
#data_path = "C:\\Users\\Basile\\Downloads\\Jérémie\\BBC-projet-lexical-analysis\\BBC-projet-lexical-analysis\\data\\"
data_file = "geneDefinition_dev.dat"
gen_id_file = "gen_id_dev.dat"
#data_file = "geneDefinition_prod.dat"

In [ ]:
# Préparation des tableaux de données
gene_code = []
gene_ids = []


for key in genes_dict:
    gene_code.append(genes_dict[key])
    gene_ids.append(key)

In [ ]:
# Read the file with gi
fd = codecs.open(data_path + gen_id_file, 'r', 'utf8')
str_gi_ids = fd.read()
fd.close()

gi_ids = str_gi_ids.split(LINE_SEPARATOR)
gi_ids = gi_ids[:len(gi_ids)-1]
    
gi_ids

In [ ]:
# Récupération et écriture de la documentation(Entrezgene_summary) de chacun des gènes avec le gi
gen_datas_summary = []

count = 0
reset = 0
fd = codecs.open(data_path + data_file, 'wb', 'utf8')
tmp = datas[0][0] + CONETENT_SEPARATOR + datas[0][1] + CONETENT_SEPARATOR + datas[0][2] + LINE_SEPARATOR
fd.write(tmp)
while count < len(gi_ids):
    try:
        gi_id = gi_ids[count]
        if gi_id != '3310':
            handle = Entrez.efetch(db="gene", id=gi_id, retmode="xml")
            records = Entrez.read(handle)
            tmp = gene_ids[count] + CONETENT_SEPARATOR + gene_code[count] + CONETENT_SEPARATOR + records[0]['Entrezgene_summary'] + LINE_SEPARATOR
            fd.write(tmp)
        else:
            tmp = gene_ids[count] + CONETENT_SEPARATOR + gene_code[count] + CONETENT_SEPARATOR + " " + LINE_SEPARATOR
            fd.write(tmp)
        print(count)
        count += 1
        reset = 0
    except Exception as e:
        if str(e) == "'Entrezgene_summary'":
            count += 1
            tmp = gene_ids[count] + CONETENT_SEPARATOR + gene_code[count] + CONETENT_SEPARATOR + " " + LINE_SEPARATOR
            fd.write(tmp)
            print(str(e))
            continue
        reset += 1
        print("Sleep", reset, e)
        time.sleep(3)
    
fd.close()


### Analyse lexicale
Nous avons sélectionné plusieurs mots liés au cancer colorectal. Nous avons utilisé la base de données MeSH pour obtenir ces mots.

In [9]:
WORD_TO_SEARCH = ['cancer', 'colorectal', 'neoplasms', 'nonpolyposis', 'lynch' ]
query_vector = [1 for i in range(len(WORD_TO_SEARCH))]


In [13]:
# Lecutre du fichier
fd = codecs.open(data_path + data_file, 'r', 'utf8')
war = fd.read()
fd.close()

In [14]:
# split des datas
sents = war.split(LINE_SEPARATOR)
# Supression de la dernière ligne vide
sents = sents[:len(sents)-1]
id_gb_texts = []
for sent in sents:
    dat = sent.split(CONETENT_SEPARATOR)

    # Zone pour appliquer des fitres / lema / nomalisation
    dat[2] = dat[2].lower()
    id_gb_texts.append(dat)

len(id_gb_texts)

37023

In [15]:
print(id_gb_texts[:10])

[['ID', 'GB_ACC', 'text'], ['1007_s_at', 'U48705', 'receptor tyrosine kinases play a key role in the communication of cells with their microenvironment. these kinases are involved in the regulation of cell growth, differentiation and metabolism. the protein encoded by this gene belongs to a subfamily of tyrosine kinase receptors with homology to dictyostelium discoideum protein discoidin i in their extracellular domain, and that are activated by various types of collagen. expression of this protein is restricted to epithelial cells, particularly in the kidney, lung, gastrointestinal tract, and brain. in addition, it has been shown to be significantly overexpressed in several human tumors. alternatively spliced transcript variants encoding different isoforms have been described for this gene. [provided by refseq, feb 2011]'], ['1053_at', 'M87338', 'this gene encodes a member of the activator 1 small subunits family. the elongation of primed dna templates by dna polymerase delta and epsi

In [16]:
# Création des collexion de texte à partir des documents
text_arr = []
for id_gb_text in id_gb_texts[1:]:
    text_arr.append(id_gb_text[2])
    
text_collection = TextCollection(text_arr)
len(text_arr)

37022

In [17]:
# Calcul du TF-IDF pour chacun des documents
tfidf_vector = [['id', 'tf-idf vector']]

for id_gb_text in id_gb_texts[1:]:
    vec = []
    for word in WORD_TO_SEARCH:
        tfidf = text_collection.tf_idf(word,id_gb_text[2] )
        vec.append(tfidf)
    tfidf_vector.append([id_gb_text[0], vec])
        
len(tfidf_vector)

37023

In [18]:
# Permet de calculer la similarité entre 2 vecteurs (requête et document)
#http://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/
def square_rooted(x):
    return round(sqrt(sum([a*a for a in x])),3)

def cosine_similarity(x,y):
    try:
        numerator = sum(a*b for a,b in zip(x,y))
        denominator = square_rooted(x)*square_rooted(y)
        return round(numerator/float(denominator),3)
    except:
        return 0.0
    

In [29]:
# Calcule de la similarité entre tous les documents et la requête
gene_similartites = [['id', 'cosSimilarity']]
for gene in tfidf_vector[1:]:
    dat = [gene[0], cosine_similarity(gene[1], query_vector)]
    gene_similartites.append(dat)
    
len(gene_similartites)

37023

In [20]:
# Filtre tous les gènes dont la similarité est de 0
def filter_zero(array):
    result = []
    for xs in array:
        if xs[1] != 0.0:
            result.append(xs)
            
    return result

In [30]:
# Filtre et trie tous les gènes
gene_similartites_filtered1 = [['id', 'cosSimilarity']]
# en
tmp = (gene_similartites[1:])
tmp.sort(key=lambda x: x[1],  reverse=True )
gene_similartites_filtered1 += tmp
len(gene_similartites_filtered1)

len(gene_similartites_filtered1)

37023

In [22]:
# Sauvegarde des 400 meilleurs gènes
np.save('gene_similartites_400', gene_similartites_filtered1[0:400])

In [23]:
# Lecture des 400 meilleurs gènes
data = np.load('gene_similartites_400.npy')

In [35]:
data[0:3]

array([['id', 'cosSimilarity'],
       ['209805_at', '0.876'],
       ['1554742_at', '0.836']], dtype='<U13')